# 課題1

別途支給するテスト用画像(136枚)から、ナンバープレートを検出するプログラムを開発してください。

#### 目標

ナンバープレートを検出する学習モデルの生成

#### 手順

##### 1. 訓練用データを配置する

画像(.jpg)とアノテーション(.txt)のデータセットを訓練用(train)と評価用(val)に分割して本ディレクトリに配置する。


```
datasets
├　images
│　├　train
│　│　└　画像(.jpg)
│　└　val
│　 　└　YOLO形式(.txt)
└　labels
 　├　train
 　│　└　画像(.jpg)
 　└　val
 　 　└　YOLO形式(.txt)
```

##### 2. 訓練する

直下セル実行

重みファイルがyolov5/runs/train/license_plate_detector/weight/に生成される。

In [1]:
!python yolov5/train.py --img 640 --batch 16 --epochs 100 --data dataset.yml --weights yolov5/yolov5m.pt --name license_plate_detector

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
train: weights=yolov5/yolov5m.pt, cfg=, data=dataset.yml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=yolov5/data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=license_plate_detector, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjs

# 課題2

検出したナンバープレートにBounding Boxを描画した画像を出力してください。

#### 目標

訓練済みモデルを使用したBounding Boxの推定

元画像にBounding Boxを描画

#### 手順

##### 0. 課題1のコマンドを実行して、モデルを訓練する

##### 1. テストデータを配置する

Bounding Boxを描画したい画像を配置

```
tests
└　input
 　└　画像(.jpg)
```

##### 2. Bounding Boxを描画する

直下セル実行

出力先のディレクトリ

```
tests
└　output
 　└　画像(.jpg)
```

In [2]:
import math
import torch
import cv2
import glob
import os

# 訓練済みモデルのインスタンス化
model = torch.hub.load(
    "yolov5", # モデルを読み込むディレクトリ
    "custom", # モデルの名前
    path="yolov5/runs/train/license_plate_detector/weights/best.pt", # 重みファイルのパス
    source="local" # gitの公開モデルではなく、ローカルのモデルを使用する
)

# 検出しきい値
model.conf = 0.5

# データアクセス
input_img_data_path = "tests/input"
output_img_data_path = "tests/output"

# 各画像への処理
# Bounding Boxの計算と描画
for data_path in glob.glob("{0}/*".format(input_img_data_path)):
    file_name = os.path.splitext(os.path.basename(data_path))[0]
    img = cv2.imread(data_path)
    result = model(img)
    for row in result.pandas().xyxy[0].itertuples():
        cv2.rectangle(
            img, 
            (math.floor(row.xmin), math.floor(row.ymin)), # Bounding Boxの左上座標
            (math.floor(row.xmax), math.floor(row.ymax)), # Bounding Boxの右下座標
            color=(255, 255, 0), # 青クリーム色の枠線
            thickness=2 # 枠線の太さ
        )
    cv2.imwrite("{0}/{1}.jpg".format(output_img_data_path, file_name), img)

YOLOv5 🚀 v7.0-378-g2f74455a Python-3.11.10 torch-2.4.1+cu124 CUDA:0 (NVIDIA RTX A4000, 16102MiB)

Fusing layers... 
Model summary: 267 layers, 46108278 parameters, 0 gradients, 107.6 GFLOPs
Adding AutoShape... 
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.

# 課題3

黄色のナンバープレート(軽自動車)の件数を出力してください。

#### 目標

訓練モデルを使用したBounding Boxの推定

Bouding Box内の代表色を取得

ナンバープレートの色の候補からRGBに最も近いものを判定


#### 手順

##### 0. 課題1のコマンドを実行して、モデルを訓練する

##### 1. テストデータを配置する

Bounding Boxを描画したい画像を配置

```
tests
└　input
 　└　画像(.jpg)
```

##### 2. 黄色のナンバープレートを数える

直下セル実行

黄色のナンバープレートの数が出力される。

In [13]:
import json

category_file = "license_plate_category/category.json"

with open(category_file, "r") as f:
    categories = json.load(f)

"""
ナンバープレートのカテゴリを判定する
@param a_score LAB空間色A値
@param b_score LAB空間色B値
@return カテゴリのインデックス
"""
def judge_license_plate_category(a_score, b_score):
    min_ix = -1 # 最小ユークリッド距離を記録した[配列のインデックス]
    min_euclidean_distance = float('inf') # 
    for ix, category in enumerate(categories):
        # ユークリッド距離
        euclidean_distance = (category['a_score'] - a_score) ** 2 + (category['b_score'] - b_score) ** 2
        if euclidean_distance < min_euclidean_distance:
            min_euclidean_distance = euclidean_distance
            min_ix = ix
    return min_ix

# 削除予定
def str_(ix):
    return categories[ix]['name']

In [14]:
import torch
import math
import numpy as np


# 訓練済みモデル
model = torch.hub.load(
    "yolov5", # モデルを読み込むディレクトリ
    "custom", # モデルの名前
    path="yolov5/runs/train/license_plate_detector/weights/best.pt", # 重みファイルのパス
    source="local" # gitの公開モデルではなく、ローカルのモデルを使用する
)

# 検出しきい値
model.conf = 0.5

# データアクセス
input_img_data_path = "tests/input"
output_img_data_path = "tests/output"

# 各ナンバープレートのカテゴリ
plate_categories_record = []

# Bounding Box計算とBox中心のRGB推定
for data_path in glob.glob("{0}/*".format(input_img_data_path)):
    img = cv2.imread(data_path)
    result = model(img) # Bounding Boxの計算

    # LAB空間色としてndarray化
    img_lab = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
    for row in result.pandas().xywh[0].itertuples():
        # ナンバープレート幅・高さの半分の領域のくり抜き
        x0 = math.floor(row.xcenter - row.width / 2)
        y0 = math.floor(row.ycenter - row.height / 2)
        x1 = math.floor(row.xcenter + row.width / 2)
        y1 = math.floor(row.ycenter + row.height / 2)
        extract_area = img_lab[y0:y1, x0:x1]

        # 領域の平均の色を取得し、代表色とする
        _, a_score, b_score = np.mean(extract_area, axis=(0, 1))

        # 代表色からナンバープレートの種類を推定する
        plate_category = judge_license_plate_category(a_score, b_score)
        print("{0}: {1}".format(data_path, str_(plate_category)))
        plate_categories_record.append(plate_category)

YOLOv5 🚀 v7.0-378-g2f74455a Python-3.11.10 torch-2.4.1+cu124 CUDA:0 (NVIDIA RTX A4000, 16102MiB)

Fusing layers... 
Model summary: 267 layers, 46108278 parameters, 0 gradients, 107.6 GFLOPs
Adding AutoShape... 
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.

tests/input/129.jpg: normal
tests/input/126.jpg: normal
tests/input/126.jpg: light
tests/input/49.jpg: normal
tests/input/49.jpg: normal
tests/input/49.jpg: normal
tests/input/40.jpg: normal
tests/input/40.jpg: light
tests/input/50.jpg: normal
tests/input/50.jpg: normal
tests/input/50.jpg: light
tests/input/128.jpg: normal
tests/input/136.jpg: normal
tests/input/78.jpg: normal
tests/input/78.jpg: light
tests/input/78.jpg: light
tests/input/46.jpg: normal
tests/input/46.jpg: normal


/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `to

tests/input/122.jpg: normal
tests/input/101.jpg: normal
tests/input/135.jpg: normal
tests/input/91.jpg: normal
tests/input/91.jpg: light
tests/input/34.jpg: normal
tests/input/100.jpg: normal
tests/input/100.jpg: light
tests/input/134.jpg: normal
tests/input/134.jpg: light
tests/input/131.jpg: normal


/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `to

...

tests/input/25.jpg: normal
tests/input/25.jpg: light
tests/input/48.jpg: normal
tests/input/48.jpg: normal
tests/input/20.jpg: normal
tests/input/26.jpg: normal
tests/input/13.jpg: normal
tests/input/13.jpg: normal
tests/input/13.jpg: light
tests/input/22.jpg: normal
tests/input/22.jpg: normal
tests/input/22.jpg: normal
tests/input/11.jpg: normal
tests/input/53.jpg: normal


/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `to

tests/input/66.jpg: light
tests/input/18.jpg: light
tests/input/44.jpg: normal
tests/input/44.jpg: light
tests/input/4.jpg: normal
tests/input/4.jpg: normal
tests/input/43.jpg: normal
tests/input/7.jpg: normal
tests/input/6.jpg: normal
tests/input/15.jpg: normal


/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/workspace/detect_license_plate/yolov5/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `to

tests/input/2.jpg: light
tests/input/2.jpg: light
tests/input/5.jpg: normal
tests/input/10.jpg: normal
tests/input/16.jpg: normal
tests/input/14.jpg: normal
tests/input/3.jpg: normal
tests/input/3.jpg: light
tests/input/9.jpg: light
tests/input/8.jpg: normal
